## pytorch import 모음

In [3]:
import torch
import torchvision
import torch.nn as nn ## neural network 모음 (nn.Linear, nn.conv2d, Batchnorm, Loss functions 등)
import torch.optim as optim ## optimization algorithm 모음 (SGD, Adam)
import torch.nn.functional as F ## 파라미터가 필요없는 Function 모음
from torch.utils.data import DataLoader ## 데이터 세트 관리 및 미니 배치 생성을 위한 함수 모음
import torchvision.datasets as datasets ## 표준 데이터 세트 모음
import torchvision.transforms as transforms ## 데이터 세트에 적용 할 수 있는 변환 관련 함수 모음
from torch.utils.tensorboard import SummaryWriter ## tensorboard에 출력하기 위한 함수 모음
import torch.backends.cudnn as cudnn ## cudnn을 다루기 위한 값 모음
from torchsummary import summary ## summary를 통한 model의 현황을 확인 하기 위함
import torch.onnx ## model을 onnx로 변환하기 위함

----

## pytorch 셋팅 관련 코드

In [4]:
## pytorch 내부적으로 사용하는 seed 값 설정
seed = 42
torch.manual_seed(seed)

## cuda를 사용할 경우 pytorch 내부적으로 사용하는 seed 값 설정
torch.cuda.manual_seed(seed)

-----

## GPU 셋팅 관련 코드

In [5]:
## cuda가 사용 가능한 지 확인
torch.cuda.is_available()

## cuda가 사용 가능하면 device에 "cuda"를 저장하고 사용 가능하지 않으면 "cpu"를 저장한다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 멀티 GPU 사용 시 device에 "cuda"를 저장하고 사용 가능하지 않으면 "cpu"를 저장한다.
    # 아래 코드의 '0', '1', '2'는 gpu가 3개 있고 그 번호가 0,1,2인 상황의 예제
    # 만약 GPU가 5개이고 사용 가능한 것이 0, 3, 4라면 "0,3,4"라고 적으면 된다.
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(list(map(str, list(range(torch.cuda.device_count())))))

## cudnn을 사용하도록 설정. GPU를 사용하고 있으면 기본값을 True이다.
import torch.backends.cudnn as cudnn
cudnn.enabled = True

## inbuilt cudnn auto-tuner 사용 중인 hardware에 가장 적합한 알고리즘을 선택하도록 허용한다.
cudnn.benchmark = True

cuda


In [6]:
"""GPU device의 사용 가능한 메모리를 코드 상에서 확인하려면 아래 함수를 사용한다."""
## unit : byte
torch.cuda.get_device_properties("cuda:0").total_memory

## unit : mega byte
torch.cuda.get_device_properties("cuda:0").total_memory // 1e6

## unit : giga byte
torch.cuda.get_device_properties("cuda:0").total_memory // 1e9

11.0

---

## dataloader의 num_workers 지정
- pytorch를 이용하여 학습을 할 때, 데이터를 불러오는 방법으로 DataLoader(from torch.utils.data_import DataLoader)를 사용한다.
- DataLoader의 `num_workers는 cpu -> gpu로 데이터를 로드할 때 사용하는 프로세스의 갯수`를 뜻한다.
- 컴퓨터에서 병목 현상이 발생하는 대표적인 구간이 바로 I/O(input/output) 연산이다. 따라서 I/O 연산에 최대 사용할 수 있는 `코어를 적당하게 나누어 주어서 병목 현상을 제거하는 것이 전체 학습 시간을 줄일 수 있는 데 도움`이 된다.
- num_workers=0이 기본적으로 사용된다. 이 옵션의 의미는 data loading이 오직 main process에서만 발생하도록 하는 synchronous 방법을 의미한다.
- 따라서 `num_workers>0 조건이 되도록 설정하여 asynchronous 하게 data loading이 가능`해지기 때문에, GPU 연산과 병렬적으로 data loading이 가능해지게 되어 병목 문제를 개선할 수 있다.
- num_workers 갯수로는 휴리스틱하게 결정하는 데 실험적으로 접근해 본 한 사람의 의견으로는 num_workers=4*GPU이다.
- 이후 연습에서는 위에서 말한 관계식처럼 num_workers = torch.cuda.device_count()*4

### dataloader의 pin_memory
- torch.utils.data.DataLoader()를 사용할 때, 옵션으로 `pin_memory=True`라는 것이 있다. pin_memory는 고정된 메모리로 default는 False이다. 이 옵션의 의미는 `cpu -> gpu로의 메모리 복사 시 오직 main process에서만 복사가 발생하도록 하는 synchronous 방법`을 의미한다. 하드웨어 자원이 많을 때 궂이 하나의 프로세스에서만 작업하는 것은 비효율적이다.
- `pin_memory=True로 설정하면 학습 중에 cpu가 데이터를 gpu로 전달하는 속도를 향상시킨다.` 따라서 이 옵션은 gpu를 사용하여 학습할 때에는 항상 사용한다고 보면 된다.


------

### GPU 사용 시 data.cuda(non_blocking=True) 사용
- gpu를 이용하여 학습할 때, 바로 앞의 dataloader의 pin_memory 사용과 더불어 data의 .cuda(non_blocking=True)는 일반적으로 반드시 사용하는 옵션이다.
- 이 옵션은 cpu->gpu로 데이터를 전달하는 메커니즘과 연관된 옵션이다.
- 텐서 및 스토리지를 고정하면 비동기(asynchronous) GPU 복사본을 사용할 수 있다. 비동기식으로 GPU에 데이터 전달 기능을 추가하려면 non_blocking=True를 to() 또는 cuda() 호출 시 argument로 전달하면 된다.